###### tasks

<ol>
<li>for arbitrary number of timesteps (prob 50-100 year range is sufficient), for each planet in each run, parse for/calculate these values
<ol>
<li>eccentricity, pericenter </li>
</ol>
<ol>
<li>for nearest and second nearest neighbor: ratios of semi-major axis, mutual hill radii, mass </li>
</ol>
</li>
<li>over the duration tracked (3, 5, 10, then 15 timesteps), for each planet in each run, find these values
<ol>
<li>max, min, average, standard deviation for each value</li>
</ol>
</li>
<li>write this into a file (pickle?). do separate file for each different timestep increment. include run and planet numbers for easy tracking</li>
<li>looking at how they change over time for each planet and system to tell which initial conditions might be most telling</li>
</ol>

In [288]:
import pickle
import pandas as pd
import numpy as np


data = dict()

cols = ['Time','mass','a','e','i','omega','Omega','M','dist_jac','dist_cen','x','y','z','u','v','w']


for rn in range(1,2):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = pd.read_csv(r'/Users/neha/Documents/lanl/single mordasini classifier/run'+str(rn)+'/PL'+str(planetnum)+'.el', skiprows=2, delimiter=r"\s+", names=cols) 

        for time in range(0,3):
            data[rn][planetnum][time] = dict()

            for col in cols:
                data[rn][planetnum][time][col] = dict()
                try:
                    data[rn][planetnum][time][col] = pl[col][time]
                except:
                    pass

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  

varA = ['e','pericenter','jacobi','mhrC','mhrF','aC_ratio','aF_ratio','massC_ratio','massF_ratio'] # C = closer, F = farther
varB = ['e','pericenter','jacobi','mhrI','mhrO','aI_ratio','aO_ratio','massI_ratio','massO_ratio'] # I = inner, O = outer

    
for rn2 in range(1,2):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['e'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        
        if (planet == 1 or planet == 10):
            te[rn2][planet]['mhrC'] = []
            te[rn2][planet]['mhrF'] = []
            te[rn2][planet]['aC_ratio'] = []
            te[rn2][planet]['aF_ratio'] = []
            te[rn2][planet]['massC_ratio'] = []
            te[rn2][planet]['massF_ratio'] = []
            
        else:
            te[rn2][planet]['mhrI'] = []
            te[rn2][planet]['mhrO'] = []
            te[rn2][planet]['aI_ratio'] = []
            te[rn2][planet]['aO_ratio'] = []
            te[rn2][planet]['massI_ratio'] = []
            te[rn2][planet]['massO_ratio'] = []

        
        
        for timestep in range (0,3):   
            try:
                te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
                te[rn2][planet]['pericenter'].append(float(data[rn2][planet][timestep]['omega']))
                te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['dist_jac']))

                massPL = float(data[rn2][planet][timestep]['mass'])
                aPL = float(data[rn2][planet][timestep]['a'])
                

                if (planet == 1 or planet == 10):
                    if (planet == 1):
                        m1 = float(data[rn2][planet+1][timestep]['mass'])
                        a1 = float(data[rn2][planet+1][timestep]['a'])

                        m2 = float(data[rn2][planet+2][timestep]['mass'])
                        a2 = float(data[rn2][planet+2][timestep]['a'])

                    else:
                        m1 = float(data[rn2][planet-1][timestep]['mass'])
                        a1 = float(data[rn2][planet-1][timestep]['a'])

                        m2 = float(data[rn2][planet-2][timestep]['mass'])
                        a2 = float(data[rn2][planet-2][timestep]['a'])

                    te[rn2][planet]['mhrC'].append((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                    te[rn2][planet]['mhrF'].append(float((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2)))
                    te[rn2][planet]['aC_ratio'].append(float(aPL/a1))
                    te[rn2][planet]['aF_ratio'].append(float(aPL/a2))
                    te[rn2][planet]['massC_ratio'].append(float(massPL/m1))
                    te[rn2][planet]['massF_ratio'].append(float(massPL/m2))


                else:
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet+1][timestep]['mass'])
                    a2 = float(data[rn2][planet+1][timestep]['a'])

                    te[rn2][planet]['mhrI'].append(float(((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                    te[rn2][planet]['mhrO'].append(float(((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))
                    te[rn2][planet]['aI_ratio'].append(float(aPL/a1))
                    te[rn2][planet]['aO_ratio'].append(float(aPL/a2))
                    te[rn2][planet]['massI_ratio'].append(float(massPL/m1))
                    te[rn2][planet]['massO_ratio'].append(float(massPL/m2))
                    
            except:
                pass

            
            
for rn2 in range(1,2):
    for planet in range (1,11):
        if (planet == 1 or planet == 10):
            for vA in varA:
                sys[rn2][planet]['initial ' + str(vA)] = te[rn2][planet][str(vA)][0]
                sys[rn2][planet]['final ' + str(vA)] = te[rn2][planet][str(vA)][-1]
                sys[rn2][planet]['mean ' + str(vA)] = np.mean(te[rn2][planet][str(vA)])  
                sys[rn2][planet]['median ' + str(vA)] = np.median(te[rn2][planet][str(vA)])
                sys[rn2][planet]['min ' + str(vA)] = np.amin(te[rn2][planet][str(vA)])
                sys[rn2][planet]['max ' + str(vA)] = np.amax(te[rn2][planet][str(vA)])
                sys[rn2][planet]['sd ' + str(vA)] = np.std(te[rn2][planet][str(vA)])
        else:
            for vB in varB:
                sys[rn2][planet]['initial ' + str(vB)] = te[rn2][planet][str(vB)][0]
                sys[rn2][planet]['final ' + str(vB)] = te[rn2][planet][str(vB)][-1]
                sys[rn2][planet]['mean ' + str(vB)] = np.mean(te[rn2][planet][str(vB)])  
                sys[rn2][planet]['median ' + str(vB)] = np.median(te[rn2][planet][str(vB)])
                sys[rn2][planet]['min ' + str(vB)] = np.amin(te[rn2][planet][str(vB)])
                sys[rn2][planet]['max ' + str(vB)] = np.amax(te[rn2][planet][str(vB)])
                sys[rn2][planet]['sd ' + str(vB)] = np.std(te[rn2][planet][str(vB)])
           
                
                
for rn2 in range(1,2):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/single mordasini classifier/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                if "ejected" in line:
                    sys[rn2][int((cols[0])[2])]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][int((cols[0])[2])]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int((cols[4])[2])]['end time'] = float(cols[6])

    
    
print(sys)

{1: {1: {'initial e': 0.0701205551129, 'final e': 0.06903280181539999, 'mean e': 0.06961399419616666, 'median e': 0.0696886256602, 'min e': 0.06903280181539999, 'max e': 0.0701205551129, 'sd e': 0.00044719809229121595, 'initial pericenter': 75.2330957673, 'final pericenter': 76.8533219478, 'mean pericenter': 76.16220706976667, 'median pericenter': 76.4002034942, 'min pericenter': 75.2330957673, 'max pericenter': 76.8533219478, 'sd pericenter': 0.6825271386676096, 'initial jacobi': 0.847422596372, 'final jacobi': 0.822617311852, 'mean jacobi': 0.8051681986603333, 'median jacobi': 0.822617311852, 'min jacobi': 0.745464687757, 'max jacobi': 0.847422596372, 'sd jacobi': 0.04341434053142555, 'initial mhrC': 0.038658433739528514, 'final mhrC': 0.0387037709373361, 'mean mhrC': 0.03866971671335848, 'median mhrC': 0.038658433739528514, 'min mhrC': 0.038646945463210854, 'max mhrC': 0.0387037709373361, 'sd mhrC': 2.4532464901646766e-05, 'initial mhrF': 0.02448987647610315, 'final mhrF': 0.0240368

In [284]:
import pickle
import pandas as pd
import numpy as np


data = dict()

cols = ['Time','mass','a','e','i','omega','Omega','M','dist_jac','dist_cen','x','y','z','u','v','w']


for rn in range(1,2):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = pd.read_csv(r'/Users/neha/Documents/lanl/single mordasini classifier/run'+str(rn)+'/PL'+str(planetnum)+'.el', skiprows=2, delimiter=r"\s+", names=cols) 

        for time in range(0,3):
            data[rn][planetnum][time] = dict()

            for col in cols:
                data[rn][planetnum][time][col] = dict()
                try:
                    data[rn][planetnum][time][col] = pl[col][time]
                except:
                    pass

    
    
sys = dict()
te = dict()

    
for rn2 in range(1,2):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['e'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        
        if (planet == 1 or planet == 10):
            te[rn2][planet]['mhrC'] = []
            te[rn2][planet]['mhrF'] = []
            te[rn2][planet]['aC_ratio'] = []
            te[rn2][planet]['aF_ratio'] = []
            te[rn2][planet]['massC_ratio'] = []
            te[rn2][planet]['massF_ratio'] = []
            
        else:
            te[rn2][planet]['mhrI'] = []
            te[rn2][planet]['mhrO'] = []
            te[rn2][planet]['aI_ratio'] = []
            te[rn2][planet]['aO_ratio'] = []
            te[rn2][planet]['massI_ratio'] = []
            te[rn2][planet]['massO_ratio'] = []

        
        
        for timestep in range (0,3):   
            try:
                te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
                te[rn2][planet]['pericenter'].append(float(data[rn2][planet][timestep]['omega']))
                te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['dist_jac']))

                massPL = float(data[rn2][planet][timestep]['mass'])
                aPL = float(data[rn2][planet][timestep]['a'])
                

                if (planet == 1 or planet == 10):
                    if (planet == 1):
                        m1 = float(data[rn2][planet+1][timestep]['mass'])
                        a1 = float(data[rn2][planet+1][timestep]['a'])

                        m2 = float(data[rn2][planet+2][timestep]['mass'])
                        a2 = float(data[rn2][planet+2][timestep]['a'])

                    else:
                        m1 = float(data[rn2][planet-1][timestep]['mass'])
                        a1 = float(data[rn2][planet-1][timestep]['a'])

                        m2 = float(data[rn2][planet-2][timestep]['mass'])
                        a2 = float(data[rn2][planet-2][timestep]['a'])

                    te[rn2][planet]['mhrC'].append((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                    te[rn2][planet]['mhrF'].append(float((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2)))
                    te[rn2][planet]['aC_ratio'].append(float(aPL/a1))
                    te[rn2][planet]['aF_ratio'].append(float(aPL/a2))
                    te[rn2][planet]['massC_ratio'].append(float(massPL/m1))
                    te[rn2][planet]['massF_ratio'].append(float(massPL/m2))


                else:
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet+1][timestep]['mass'])
                    a2 = float(data[rn2][planet+1][timestep]['a'])

                    te[rn2][planet]['mhrI'].append(float(((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                    te[rn2][planet]['mhrO'].append(float(((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))
                    te[rn2][planet]['aI_ratio'].append(float(aPL/a1))
                    te[rn2][planet]['aO_ratio'].append(float(aPL/a2))
                    te[rn2][planet]['massI_ratio'].append(float(massPL/m1))
                    te[rn2][planet]['massO_ratio'].append(float(massPL/m2))
                    
            except:
                pass

            
for rn2 in range(1,2):
    for planet in range (1,11):
        sys[rn2][planet]['intial e'] = te[rn2][planet]['e'][0]
        sys[rn2][planet]['final e'] = te[rn2][planet]['e'][-1]
        sys[rn2][planet]['mean e'] = np.mean(te[rn2][planet]['e'])  
        sys[rn2][planet]['median e'] = np.median(te[rn2][planet]['e'])
        sys[rn2][planet]['min e'] = np.amin(te[rn2][planet]['e'])
        sys[rn2][planet]['max e'] = np.amax(te[rn2][planet]['e'])
        sys[rn2][planet]['sd e'] = np.std(te[rn2][planet]['e'])
                    
        sys[rn2][planet]['intial pericenter'] = te[rn2][planet]['pericenter'][0]
        sys[rn2][planet]['final pericenter'] = te[rn2][planet]['pericenter'][-1]
        sys[rn2][planet]['mean pericenter'] = np.mean(te[rn2][planet]['pericenter'])  
        sys[rn2][planet]['median pericenter'] = np.median(te[rn2][planet]['pericenter'])
        sys[rn2][planet]['min pericenter'] = np.amin(te[rn2][planet]['pericenter'])
        sys[rn2][planet]['max pericenter'] = np.amax(te[rn2][planet]['pericenter'])
        sys[rn2][planet]['sd pericenter'] = np.std(te[rn2][planet]['pericenter'])
        
        sys[rn2][planet]['intial jacobi'] = te[rn2][planet]['jacobi'][0]
        sys[rn2][planet]['final jacobi'] = te[rn2][planet]['jacobi'][-1]
        sys[rn2][planet]['mean jacobi'] = np.mean(te[rn2][planet]['jacobi'])  
        sys[rn2][planet]['median jacobi'] = np.median(te[rn2][planet]['jacobi'])
        sys[rn2][planet]['min jacobi'] = np.amin(te[rn2][planet]['jacobi'])
        sys[rn2][planet]['max jacobi'] = np.amax(te[rn2][planet]['jacobi'])
        sys[rn2][planet]['sd jacobi'] = np.std(te[rn2][planet]['jacobi'])
           
              
        if (planet == 1 or planet == 10):
            sys[rn2][planet]['intial mhrC'] = te[rn2][planet]['mhrC'][0]
            sys[rn2][planet]['final mhrC'] = te[rn2][planet]['mhrC'][-1]
            sys[rn2][planet]['mean mhrC'] = np.mean(te[rn2][planet]['mhrC'])  
            sys[rn2][planet]['median mhrC'] = np.median(te[rn2][planet]['mhrC'])
            sys[rn2][planet]['min mhrC'] = np.amin(te[rn2][planet]['mhrC'])
            sys[rn2][planet]['max mhrC'] = np.amax(te[rn2][planet]['mhrC'])
            sys[rn2][planet]['sd mhrC'] = np.std(te[rn2][planet]['mhrC'])
                
            sys[rn2][planet]['intial mhrF'] = te[rn2][planet]['mhrF'][0]
            sys[rn2][planet]['final mhrF'] = te[rn2][planet]['mhrF'][-1]
            sys[rn2][planet]['mean mhrF'] = np.mean(te[rn2][planet]['mhrF'])  
            sys[rn2][planet]['median mhrF'] = np.median(te[rn2][planet]['mhrF'])
            sys[rn2][planet]['min mhrF'] = np.amin(te[rn2][planet]['mhrF'])
            sys[rn2][planet]['max mhrF'] = np.amax(te[rn2][planet]['mhrF'])
            sys[rn2][planet]['sd mhrF'] = np.std(te[rn2][planet]['mhrF'])
                
            sys[rn2][planet]['intial aC_ratio'] = te[rn2][planet]['aC_ratio'][0]
            sys[rn2][planet]['final aC_ratio'] = te[rn2][planet]['aC_ratio'][-1]
            sys[rn2][planet]['mean aC_ratio'] = np.mean(te[rn2][planet]['aC_ratio'])  
            sys[rn2][planet]['median aC_ratio'] = np.median(te[rn2][planet]['aC_ratio'])
            sys[rn2][planet]['min aC_ratio'] = np.amin(te[rn2][planet]['aC_ratio'])
            sys[rn2][planet]['max aC_ratio'] = np.amax(te[rn2][planet]['aC_ratio'])
            sys[rn2][planet]['sd aC_ratio'] = np.std(te[rn2][planet]['aC_ratio'])
                
            sys[rn2][planet]['intial aF_ratio'] = te[rn2][planet]['aF_ratio'][0]
            sys[rn2][planet]['final aF_ratio'] = te[rn2][planet]['aF_ratio'][-1]
            sys[rn2][planet]['mean aF_ratio'] = np.mean(te[rn2][planet]['aF_ratio'])  
            sys[rn2][planet]['median aF_ratio'] = np.median(te[rn2][planet]['aF_ratio'])
            sys[rn2][planet]['min aF_ratio'] = np.amin(te[rn2][planet]['aF_ratio'])
            sys[rn2][planet]['max aF_ratio'] = np.amax(te[rn2][planet]['aF_ratio'])
            sys[rn2][planet]['sd aF_ratio'] = np.std(te[rn2][planet]['aF_ratio'])
                
            sys[rn2][planet]['intial massC_ratio'] = te[rn2][planet]['massC_ratio'][0]
            sys[rn2][planet]['final massC_ratio'] = te[rn2][planet]['massC_ratio'][-1]
            sys[rn2][planet]['mean massC_ratio'] = np.mean(te[rn2][planet]['massC_ratio'])  
            sys[rn2][planet]['median massC_ratio'] = np.median(te[rn2][planet]['massC_ratio'])
            sys[rn2][planet]['min massC_ratio'] = np.amin(te[rn2][planet]['massC_ratio'])
            sys[rn2][planet]['max massC_ratio'] = np.amax(te[rn2][planet]['massC_ratio'])
            sys[rn2][planet]['sd massC_ratio'] = np.std(te[rn2][planet]['massC_ratio'])
                
            sys[rn2][planet]['intial massF_ratio'] = te[rn2][planet]['massF_ratio'][0]
            sys[rn2][planet]['final massF_ratio'] = te[rn2][planet]['massF_ratio'][-1]
            sys[rn2][planet]['mean massF_ratio'] = np.mean(te[rn2][planet]['massF_ratio'])  
            sys[rn2][planet]['median massF_ratio'] = np.median(te[rn2][planet]['massF_ratio'])
            sys[rn2][planet]['min massF_ratio'] = np.amin(te[rn2][planet]['massF_ratio'])
            sys[rn2][planet]['max massF_ratio'] = np.amax(te[rn2][planet]['massF_ratio'])
            sys[rn2][planet]['sd massF_ratio'] = np.std(te[rn2][planet]['massF_ratio'])

            
        else:
            sys[rn2][planet]['intial mhrI'] = te[rn2][planet]['mhrI'][0]
            sys[rn2][planet]['final mhrI'] = te[rn2][planet]['mhrI'][-1]
            sys[rn2][planet]['mean mhrI'] = np.mean(te[rn2][planet]['mhrI'])  
            sys[rn2][planet]['median mhrI'] = np.median(te[rn2][planet]['mhrI'])
            sys[rn2][planet]['min mhrI'] = np.amin(te[rn2][planet]['mhrI'])
            sys[rn2][planet]['max mhrI'] = np.amax(te[rn2][planet]['mhrI'])
            sys[rn2][planet]['sd mhrI'] = np.std(te[rn2][planet]['mhrI'])
                
            sys[rn2][planet]['intial mhrO'] = te[rn2][planet]['mhrO'][0]
            sys[rn2][planet]['final mhrO'] = te[rn2][planet]['mhrO'][-1]
            sys[rn2][planet]['mean mhrO'] = np.mean(te[rn2][planet]['mhrO'])  
            sys[rn2][planet]['median mhrO'] = np.median(te[rn2][planet]['mhrO'])
            sys[rn2][planet]['min mhrO'] = np.amin(te[rn2][planet]['mhrO'])
            sys[rn2][planet]['max mhrO'] = np.amax(te[rn2][planet]['mhrO'])
            sys[rn2][planet]['sd mhrO'] = np.std(te[rn2][planet]['mhrO'])
                
            sys[rn2][planet]['intial aI_ratio'] = te[rn2][planet]['aI_ratio'][0]
            sys[rn2][planet]['final aI_ratio'] = te[rn2][planet]['aI_ratio'][-1]
            sys[rn2][planet]['mean aI_ratio'] = np.mean(te[rn2][planet]['aI_ratio'])  
            sys[rn2][planet]['median aI_ratio'] = np.median(te[rn2][planet]['aI_ratio'])
            sys[rn2][planet]['min aI_ratio'] = np.amin(te[rn2][planet]['aI_ratio'])
            sys[rn2][planet]['max aI_ratio'] = np.amax(te[rn2][planet]['aI_ratio'])
            sys[rn2][planet]['sd aI_ratio'] = np.std(te[rn2][planet]['aI_ratio'])
                
            sys[rn2][planet]['intial aO_ratio'] = te[rn2][planet]['aO_ratio'][0]
            sys[rn2][planet]['final aO_ratio'] = te[rn2][planet]['aO_ratio'][-1]
            sys[rn2][planet]['mean aO_ratio'] = np.mean(te[rn2][planet]['aO_ratio'])  
            sys[rn2][planet]['median aO_ratio'] = np.median(te[rn2][planet]['aO_ratio'])
            sys[rn2][planet]['min aO_ratio'] = np.amin(te[rn2][planet]['aO_ratio'])
            sys[rn2][planet]['max aO_ratio'] = np.amax(te[rn2][planet]['aO_ratio'])
            sys[rn2][planet]['sd aO_ratio'] = np.std(te[rn2][planet]['aO_ratio'])
                
            sys[rn2][planet]['intial massI_ratio'] = te[rn2][planet]['massI_ratio'][0]
            sys[rn2][planet]['final massI_ratio'] = te[rn2][planet]['massI_ratio'][-1]
            sys[rn2][planet]['mean massI_ratio'] = np.mean(te[rn2][planet]['massI_ratio'])  
            sys[rn2][planet]['median massI_ratio'] = np.median(te[rn2][planet]['massI_ratio'])
            sys[rn2][planet]['min massI_ratio'] = np.amin(te[rn2][planet]['massI_ratio'])
            sys[rn2][planet]['max massI_ratio'] = np.amax(te[rn2][planet]['massI_ratio'])
            sys[rn2][planet]['sd massI_ratio'] = np.std(te[rn2][planet]['massI_ratio'])
                
            sys[rn2][planet]['intial massO_ratio'] = te[rn2][planet]['massO_ratio'][0]
            sys[rn2][planet]['final massO_ratio'] = te[rn2][planet]['massO_ratio'][-1]
            sys[rn2][planet]['mean massO_ratio'] = np.mean(te[rn2][planet]['massO_ratio'])  
            sys[rn2][planet]['median massO_ratio'] = np.median(te[rn2][planet]['massO_ratio'])
            sys[rn2][planet]['min massO_ratio'] = np.amin(te[rn2][planet]['massO_ratio'])
            sys[rn2][planet]['max massO_ratio'] = np.amax(te[rn2][planet]['massO_ratio'])
            sys[rn2][planet]['sd massO_ratio'] = np.std(te[rn2][planet]['massO_ratio'])    
                    
                
                
for rn2 in range(1,2):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/single mordasini classifier/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                if "ejected" in line:
                    sys[rn2][int((cols[0])[2])]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][int((cols[0])[2])]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int((cols[4])[2])]['end time'] = float(cols[6])

    
    
print(sys)

{1: {1: {'intial e': 0.0701205551129, 'final e': 0.06903280181539999, 'mean e': 0.06961399419616666, 'median e': 0.0696886256602, 'min e': 0.06903280181539999, 'max e': 0.0701205551129, 'sd e': 0.00044719809229121595, 'intial pericenter': 75.2330957673, 'final pericenter': 76.8533219478, 'mean pericenter': 76.16220706976667, 'median pericenter': 76.4002034942, 'min pericenter': 75.2330957673, 'max pericenter': 76.8533219478, 'sd pericenter': 0.6825271386676096, 'intial jacobi': 0.847422596372, 'final jacobi': 0.822617311852, 'mean jacobi': 0.8051681986603333, 'median jacobi': 0.822617311852, 'min jacobi': 0.745464687757, 'max jacobi': 0.847422596372, 'sd jacobi': 0.04341434053142555, 'intial mhrC': 0.038658433739528514, 'final mhrC': 0.0387037709373361, 'mean mhrC': 0.03866971671335848, 'median mhrC': 0.038658433739528514, 'min mhrC': 0.038646945463210854, 'max mhrC': 0.0387037709373361, 'sd mhrC': 2.4532464901646766e-05, 'intial mhrF': 0.02448987647610315, 'final mhrF': 0.024036805766

In [1]:
import pandas as pd
import numpy as np
import csv


data = dict()

cols = ['Time','mass','a','e','i','omega','Omega','M','dist_jac','dist_cen','x','y','z','u','v','w']


for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = pd.read_csv(r'/Users/neha/Documents/lanl/single mordasini classifier/run'+str(rn)+'/PL'+str(planetnum)+'.el', skiprows=2, delimiter=r"\s+", names=cols) 

        for time in range(0,20):
            data[rn][planetnum][time] = dict()

            for col in cols:
                data[rn][planetnum][time][col] = dict()
                try:
                    data[rn][planetnum][time][col] = pl[col][time]
                except:
                    pass

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  

var = ['e','pericenter','jacobi','mhr1','mhr2','a1_ratio','a2_ratio','mass1_ratio','mass2_ratio'] 
# for pl 1 & 10, next and next next (first and second over neighboring) planets. for rest, inner and outer

    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['e'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr1'] = []
        te[rn2][planet]['mhr2'] = []
        te[rn2][planet]['a1_ratio'] = []
        te[rn2][planet]['a2_ratio'] = []
        te[rn2][planet]['mass1_ratio'] = []
        te[rn2][planet]['mass2_ratio'] = []

    
        for timestep in range (0,20):
            try:
                te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
                te[rn2][planet]['pericenter'].append(float(data[rn2][planet][timestep]['omega']))
                te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['dist_jac']))

                massPL = float(data[rn2][planet][timestep]['mass'])
                aPL = float(data[rn2][planet][timestep]['a'])
                
                
                if (planet == 1):
                    m1 = float(data[rn2][planet+1][timestep]['mass'])
                    a1 = float(data[rn2][planet+1][timestep]['a'])

                    m2 = float(data[rn2][planet+2][timestep]['mass'])
                    a2 = float(data[rn2][planet+2][timestep]['a'])

                elif (planet == 10):
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet-2][timestep]['mass'])
                    a2 = float(data[rn2][planet-2][timestep]['a'])
                        
                else:
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet+1][timestep]['mass'])
                    a2 = float(data[rn2][planet+1][timestep]['a'])

                    
                te[rn2][planet]['mhr1'].append((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                te[rn2][planet]['mhr2'].append(float((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2)))
                te[rn2][planet]['a1_ratio'].append(float(aPL/a1))
                te[rn2][planet]['a2_ratio'].append(float(aPL/a2))
                te[rn2][planet]['mass1_ratio'].append(float(massPL/m1))
                te[rn2][planet]['mass2_ratio'].append(float(massPL/m2))
                
                
            except:
                pass

            
            
for rn2 in range(1,101):
    for planet in range (1,11):
        for v in var:
            sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
            sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
            sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
            sys[rn2][planet]['median ' + str(v)] = np.median(te[rn2][planet][str(v)])
            sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
            sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
            sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
           
                
                
for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/single mordasini classifier/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                if "ejected" in line:
                    sys[rn2][int((cols[0])[2])]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][int((cols[0])[2])]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int((cols[4])[2])]['end time'] = float(cols[6])

In [2]:
import pickle

with open('ts20.pickle', 'wb') as p:
    pickle.dump(sys, p)

In [3]:
# want csv file to be like a table? columns: run and planet numbers, all 63 data entires. rows: 1000, one for each pl


import pickle        
import base64
import csv

your_pickle_obj = pickle.loads(open('ts3.pickle', 'rb').read())
with open('ts3test.csv', 'a', encoding='utf8') as csv_file:
    wr = csv.writer(csv_file, delimiter=',')
    pickle_bytes = pickle.dumps(your_pickle_obj)            # unsafe to write
    b64_bytes = base64.b64encode(pickle_bytes)  # safe to write but still bytes
    b64_str = b64_bytes.decode('utf8')          # safe and in utf8
    wr.writerow(['col1', 'col2', b64_str])

In [18]:
for rn2 in range(1,5):
    with open('ts3.csv', 'w') as f:
        my = sys[rn2]
        for key in my.keys():
            f.write("%s, %s\n" % (rn2, sys[rn2][pl]))

            
# with open('ts3testsfs.csv', 'w') as f:
#     w = csv.DictWriter(f, sys[rn2][pl])
#     w.writeheader()
#     w.writerow(sys)

TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed

In [ ]:
csv_columns = ['No','Name','Country']

csv_file = "jskdfl.csv"
for rn2 in range (1,2):
    for planet in range(1,11):
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=sys[rn2][planet])
            writer.writeheader()
            for data in dict_data:
                writer.writerow(data)